In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
PATH = '/content/drive/MyDrive/Parallel_PDE_project/fourier_neural_operator'
os.chdir(PATH)
import sys  
sys.path.insert(0, '/content/drive/MyDrive/Parallel_PDE_project/fourier_neural_operator/Notebooks')

In [ ]:
from utils import *
from Adam import * 
from FNO import FNO

In [ ]:
################################################################
# load data and data normalization
################################################################
#EqFile = 'Burgers'
EqFile = 'DarcyFlow'
PDE_dir =  os.path.join(PATH, 'data', EqFile)
x_train = torch.load( os.path.join(PDE_dir, f'{EqFile}_x_train.pt'))
y_train = torch.load(os.path.join(PDE_dir, f'{EqFile}_y_train.pt'))
x_test = torch.load(os.path.join(PDE_dir, f'{EqFile}_x_test.pt'))
y_test= torch.load(os.path.join(PDE_dir, f'{EqFile}_y_test.pt'))

In [ ]:
#Parameters
ntrain = x_train.shape[0]
ntest = x_test.shape[0]
s = x_test.shape[-2]
batch_size = 10

In [ ]:
x_normalizer = UnitGaussianNormalizer(x_train)
x_train = x_normalizer.encode(x_train)
x_test = x_normalizer.encode(x_test)

y_normalizer = UnitGaussianNormalizer(y_train)
y_train = y_normalizer.encode(y_train)

x_train = x_train.reshape(ntrain,s,s,1)
#x_train = x_train.reshape(ntrain,s,1)
x_test = x_test.reshape(ntest,s,s,1)
#x_test = x_test.reshape(ntest,s,1)

In [ ]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

# Costumized Lifting and Projection maps. 
It can be also ignored and used the predesigned maps 
in the files.

In [ ]:
################################################################
# Lifting map
################################################################

class R(nn.Module):
  def __init__(self, input, width):
    super().__init__()
    self.fc1 = nn.Linear(input, width)
  def forward(self,x):
    x = self.fc1(x)
    return x

################################################################
# Projection map
################################################################

class P(nn.Module):
  def __init__(self, width1, width2):
    super().__init__()
    self.fc1 = nn.Linear(width1, 64)
    self.fc2 = nn.Linear(64, 64)
    self.fc3 = nn.Linear(64, width2)
    self.nonlinear =torch.nn.functional.relu
  def forward(self,x):
    x = self.fc1(x)
    x = self.nonlinear(x)
    x = self.fc2(x)
    x = self.nonlinear(x)
    x = self.fc3(x)
    return x

# FNO

In [ ]:
#NN parameters
activ_vec = ['relu', 'tanh', 'sine', 'gelu']
learning_rate = 0.001
activ = activ_vec[0]
epochs = 100
step_size = epochs//10
gamma = 0.5
layers = 8
# wavenumber = [25, 25]
wavenumber =[20, 20]
features_ = 32

In [ ]:
loss_train = []
loss_test =  []
epoch_vec = []
filename = f'FNO_problem_{EqFile}_epoch_{epochs}_wavenumber_{wavenumber}_features_{features_}_act_{activ}_layers_{layers}'
filename

'FNO_problem_DarcyFlow_epoch_100_wavenumber_[20, 20]_features_32_act_relu_layers_8'

In [ ]:
model = FNO(wavenumber = wavenumber,
            features_ =features_, 
            activation = activ,
            layers =layers, 
            lifting = R,
            proj = P).to(device)
# model =  torch.load(os.path.join('trained_models',filename+'.plt'))

# Training

In [ ]:
print(count_params(model))

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, steps_per_epoch=len(train_loader), epochs=epochs)
scheduler= torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
myloss = LpLoss(size_average=False)
y_normalizer.to(device)

################################################################
# training and evaluation
for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_l2 = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x).reshape(batch_size, s, s)
        #out = model(x).reshape(batch_size, s)
        out = y_normalizer.decode(out)
        y = y_normalizer.decode(y)

        loss = myloss(out.view(batch_size,-1), y.view(batch_size,-1))
        loss.backward()

        optimizer.step()
        train_l2 += loss.item()
    
    epoch_vec.append(ep)
    #e= epoch_vec[-1]
    #epoch_vec.append(e+1)
    
    model.eval()
    test_l2 = 0.0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
    
            out = model(x).reshape(batch_size, s, s)
            #out = model(x).reshape(batch_size, s)
            out = y_normalizer.decode(out)

            test_l2 += myloss(out.view(batch_size,-1), y.view(batch_size,-1)).item()

    train_l2/= ntrain
    test_l2 /= ntest

    loss_train.append(train_l2)
    loss_test.append(test_l2)

    t2 = default_timer()
    print(ep, t2-t1, train_l2, test_l2)
print(sum(loss_test[-5:])/5)

6568513
0 4.701346278000074 0.1656050670146942 0.11375909566879272
1 4.68923454999981 0.1015530566573143 0.09762178599834442
2 4.68265253200002 0.08269901037216186 0.07816955983638764
3 4.677410849999887 0.06560284325480462 0.059709425270557406
4 4.688932884999986 0.049409393787384036 0.047282481491565706
5 4.69249793899985 0.03847777059674263 0.03967985302209854
6 4.689118606999955 0.03338325387239456 0.03544019967317581
7 4.6908645290000095 0.03104275654256344 0.036285606324672696
8 4.6811617079999905 0.028759758219122886 0.02859650954604149
9 4.68241299400006 0.025943032294511795 0.030839163064956664
10 4.685051736000105 0.02423291416466236 0.02672997459769249
11 4.690364655000167 0.02152752083539963 0.02570570006966591
12 4.684576457000048 0.02063585987687111 0.0268234720826149
13 4.6966224100001455 0.01972132296860218 0.025250989645719528
14 4.692921833000128 0.019438393130898474 0.024709376245737075
15 4.688525662000075 0.019723172336816787 0.02320523262023926
16 4.68797530100005

In [ ]:
print(f'saving model at epoch {ep}')
file_path = os.path.join(PATH, 'trained_models')
torch.save(model, os.path.join(file_path, filename+'.plt'))

In [ ]:
  fig, axs = plt.subplots(2, 1, figsize=(17,17))
  axs[0].plot(epoch_vec, loss_train)
  axs[0].set_title(f'Training loss {filename}')
  axs[0].set_yscale('log')
  axs[1].plot(epoch_vec, loss_test, 'tab:orange')
  axs[1].set_title('Test loss')
  axs[1].set_yscale('log')
  plt.savefig(os.path.join(PATH, 'figures', filename+'.png'))